In [31]:
import pandas as pd
import numpy as np
from datetime import date, datetime

Test to see if pandas is working


In [2]:
pd.__version__

'1.5.3'

Load data from csv. ri stands for Rhode Island


In [170]:
ri = pd.read_csv("police.csv")

Take a quick look at what data we get.


In [69]:
display(ri.head())
display(ri.dtypes)

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


stop_date              object
stop_time              object
county_name           float64
driver_gender          object
driver_age_raw        float64
driver_age            float64
driver_race            object
violation_raw          object
violation              object
search_conducted         bool
search_type            object
stop_outcome           object
is_arrested            object
stop_duration          object
drugs_related_stop       bool
dtype: object

The data types of some columns are inaccurate. We can use astype() to reset the column dtypes.


In [67]:
column_dtypes = {
    'stop_date': 'datetime64[D]',
    'driver_age_raw': 'Int32',
    'driver_age': 'Int32'
}

ri = ri.astype(column_dtypes)
ri.dtypes

stop_date             datetime64[ns]
stop_time                     object
county_name                  float64
driver_gender                 object
driver_age_raw                 Int32
driver_age                     Int32
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                   object
stop_duration                 object
drugs_related_stop              bool
dtype: object

## 1. Remove columns that only contain missing values

Data cleanup work needs done before any indepth analysis; otherwise it'll be a waste of time.


In [18]:
display(ri.shape)
ri.isnull().sum()

(91741, 15)

stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5335
driver_age_raw         5327
driver_age             5621
driver_race            5333
violation_raw          5333
violation              5333
search_conducted          0
search_type           88545
stop_outcome           5333
is_arrested            5333
stop_duration          5333
drugs_related_stop        0
dtype: int64

Now that county_name contains only NaA, this column is not needed and should be removed.


In [14]:
ri.drop('county_name', axis=1, inplace=True)
ri.head()

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91736,2015-12-31,20:27,M,1986.0,29.0,White,Speeding,Speeding,False,NaN,Warning,False,0-15 Min,False
91737,2015-12-31,20:35,F,1982.0,33.0,White,Equipment/Inspection Violation,Equipment,False,NaN,Warning,False,0-15 Min,False
91738,2015-12-31,20:45,M,1992.0,23.0,White,Other Traffic Violation,Moving violation,False,NaN,Warning,False,0-15 Min,False
91739,2015-12-31,21:42,M,1993.0,22.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


In [15]:
ri.isnull().sum()

stop_date                 0
stop_time                 0
driver_gender          5335
driver_age_raw         5327
driver_age             5621
driver_race            5333
violation_raw          5333
violation              5333
search_conducted          0
search_type           88545
stop_outcome           5333
is_arrested            5333
stop_duration          5333
drugs_related_stop        0
dtype: int64

An alternative way of removing null columns, also a simpler way, is using dropna() method.

The value of how parameter can be "all" or "any" (default). Make sure set how = "all".


In [28]:
ri.dropna(axis="columns", how="all", inplace=True)
ri.shape

(91741, 14)

Do Men or Women speed more often?


In [120]:
for gender in ['M', 'F']:
    print("{0}:\n{1}\n".format(
        gender, ri[ri.driver_gender == gender].violation.value_counts(normalize=True)))

M:
Speeding               0.524350
Moving violation       0.207012
Equipment              0.135671
Other                  0.057668
Registration/plates    0.038461
Seat belt              0.036839
Name: violation, dtype: float64

F:
Speeding               0.658500
Moving violation       0.136277
Equipment              0.105780
Registration/plates    0.043086
Other                  0.029348
Seat belt              0.027009
Name: violation, dtype: float64



In [132]:
ri.groupby('driver_gender').violation.value_counts(
    normalize=True)[:, 'Speeding']

driver_gender
F    0.65850
M    0.52435
Name: violation, dtype: float64

Below we use groupby() to create a pd.Series with two levels of indices using a MultiIndex.

Then we call the unstack() method on the series to convert it into a matrix.

The unstack() function reshapes the series into a DataFrame, where the first level of the index becomes the row index, and the second level of the index becomes the column index.


In [141]:
sr = ri.groupby(['violation', 'driver_gender']).search_conducted.mean()
df = sr.unstack()
display(df, type(df))

driver_gender,F,M
violation,,
Equipment,0.042622,0.070081
Moving violation,0.036205,0.059831
Other,0.056522,0.047146
Registration/plates,0.066140,0.110376
Seat belt,0.012598,0.037980
Speeding,0.008720,0.024925


pandas.core.frame.DataFrame

## 2. How often do drivers frisk?


In [171]:
ri.dropna(axis='columns', how='all', inplace=True)

In [153]:
ri['frisk'] = ri.search_type.str.contains('Frisk')

In [165]:
ri.frisk.mean()

0.08573216520650813

In [172]:
ri.head()

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


In [176]:
ri['stop_datetime'] = ri.stop_date.str.cat(
    ri.stop_time, sep=' ').astype('datetime64')

In [179]:
ri.dtypes

stop_date                     object
stop_time                     object
driver_gender                 object
driver_age_raw               float64
driver_age                   float64
driver_race                   object
violation_raw                 object
violation                     object
search_conducted                bool
search_type                   object
stop_outcome                  object
is_arrested                   object
stop_duration                 object
drugs_related_stop              bool
stop_datetime         datetime64[ns]
dtype: object

In [180]:
ri.head()

,stop_date,stop_time,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,stop_datetime
0,2005-01-02,01:55,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,2005-01-02 01:55:00
1,2005-01-18,08:15,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,2005-01-18 08:15:00
2,2005-01-23,23:15,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,2005-01-23 23:15:00
3,2005-02-20,17:15,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False,2005-02-20 17:15:00
4,2005-03-14,10:00,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False,2005-03-14 10:00:00


In [182]:
ri.drugs_related_stop.value_counts(dropna=False)

False    90926
True       815
Name: drugs_related_stop, dtype: int64

In [198]:
print("{:.2%}".format(ri.drugs_related_stop.mean()))

0.89%


In [215]:
print("%.2f%%" % (ri.drugs_related_stop.mean() * 100))

0.89%


In [226]:
import cufflinks as cf
cf.go_offline(connected=True)

ri.groupby(ri.stop_datetime.dt.hour).drugs_related_stop.mean().iplot()

In [227]:
import plotly.express as px

data = ri.groupby(ri.stop_datetime.dt.hour).drugs_related_stop.mean()

fig = px.line(data)
fig.show()

In [228]:
ri.stop_duration.value_counts()

0-15 Min     69543
16-30 Min    13635
30+ Min       3228
2                1
1                1
Name: stop_duration, dtype: int64

In [231]:
ri.loc[ri.stop_duration.isin(['1', '2']), 'stop_duration'] = 'NaN'

In [232]:
ri.stop_duration.value_counts()

0-15 Min     69543
16-30 Min    13635
30+ Min       3228
NaN              2
Name: stop_duration, dtype: int64

In [245]:
ri.loc[ri.stop_duration == 'NaN', 'stop_duration'] = np.nan

In [248]:
ri.stop_duration.value_counts(dropna=False, normalize=True)

0-15 Min     0.758028
16-30 Min    0.148623
NaN          0.058163
30+ Min      0.035186
Name: stop_duration, dtype: float64